In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy
import sparse
from helpers import _generate_dist, _lesion_stats, _testFun1d

#%matplotlib inline

In [ ]:
def _contour_slices(_slice, cmap="gray", total_levels=6, view='sagital'):
    # This function includes also plotting contour plot
    # link --> https://www.python-course.eu/matplotlib_contour_plot.php
    # link --> https://github.com/silx-kit/silx/issues/2242
    
    fig, axes = plt.subplots(1, 1, figsize=(16,16))
    contour_levels = total_levels
    #['sagital', 'coronal', 'axial'] --> view
    
    
    row, col = np.shape(_slice)
    y = np.arange(0, row)
    x = np.arange(0, col)
    xx, yy = np.meshgrid(x, y)


    zzmin, zzmax = np.min(_slice), np.max(_slice)
    levels = np.linspace(zzmin, zzmax, contour_levels)

    #Display image with contour plot
    #axes[i].imshow(slice.T, cmap=cmap, origin="lower")
    contour = axes.contour(yy, xx, _slice, levels)
    axes.clabel(contour, colors = 'k', fmt = '%2.1f', fontsize=12)
    c = ('#ff0000', '#ffff00', '#0000FF', '0.6', 'c', 'm')
    axes.set_title(view)
    axes.contourf(yy, xx, _slice, colors=c)

        
    plt.show()
    
def _show_slices(_slice, cmap="gray", view='sagital'):
    # This function comes from the nibabel
    # tutorial --> https://nipy.org/nibabel/coordinate_systems.html#introducing-someone
    fig, axes = plt.subplots(1, 1, figsize=(15, 15))

    #['sagital', 'coronal', 'axial'] --> view
    axes.imshow(_slice.T, cmap=cmap, origin="lower")
    axes.set_title(view)    
    plt.show()

In [ ]:
MICCAI_DIR=os.path.join("/data1/local+data", 
                         "MICCAI_2016")

print("{0}".format(80 * "-"))
print("MICCAI_DIR is {0}".format(MICCAI_DIR))
print("{0}".format(80 * "-"))

In [ ]:
# Path to the dataset and lesions

dir_dataset = os.path.join("/data1/local+data/MICCAI_2016", 
                          "Preprocessed/080_013")

lesion_path = os.path.join(dir_dataset, 'lesion_registered.nii.gz')
flair_path = os.path.join(dir_dataset, 'FLAIR_preprocessed.nii.gz')
gado_path = os.path.join(dir_dataset, 'GADO_preprocessed.nii.gz')

# Loading images
lesion = nib.load(lesion_path).get_fdata()
flair = nib.load(flair_path).get_fdata()
gado = nib.load(gado_path).get_fdata()

#Normalization
norm_f = np.linalg.norm(flair)
norm_g = np.linalg.norm(gado)

flair /= norm_f
gado /= norm_g

In [ ]:
slice_0 = flair[50, :, :]
slice_1 = flair[:,220,:]
slice_2 = flair[:, :, 240]
_show_slices(slice_1)

print("{0}".format(80 * "-"))
print("flair image shape is {0}".format(flair.shape))
print("{0}\n".format(80 * "-"))

print("{0}".format(80 * "-"))
print("Showing lesions now")
print("{0}".format(80 * "-"))


slice_0 = lesion[50, :, :]
slice_1 = lesion[:,220,:]
slice_2 = lesion[:, :, 240]
_show_slices(slice_2)

In [ ]:
print("{0}".format(80 * "-"))
print("Printing data frame table now")
print("{0}".format(80 * "-"))

props_table_df, _label_lesion = _lesion_stats(lesion)

In [ ]:
_lesion_num = 1

print("{0}".format(80 * "-"))
print("Working with lesion {0}".format(_lesion_num))
print("{0}".format(80 * "-"))

_region = _label_lesion==_lesion_num
_res = np.where(_region == True)
    
# Need to crop the image
s_min = min(_res[0])
s_max = max(_res[0])
x_min = min(_res[1])
x_max = max(_res[1])
y_min = min(_res[2])
y_max = max(_res[2])

# We need to use sparse arrays
_zoomed_lesion = _region[s_min:s_max, x_min:x_max, y_min:y_max]
_edt, _inds = _generate_dist(_zoomed_lesion)

In [ ]:
#slice_0 = _label_lesion[50, :, :]
#slice_1 = _label_lesion[:,240,:]
#slice_2 = _label_lesion[:, :, 200]
slice_0 = _edt[:, 5, :]
slice_1 = _edt[:, 50, :]
slice_2 = _edt[:,:,40]
_show_slices(slice_1)

In [ ]:
_contour_slices(slice_1, 'gray')

In [ ]:
np.unique(slice_1)

In [ ]:
a = np.where((slice_1 > 0) & (slice_1 < 1.8))
b = slice_1[a]
result, _ = _testFun1d(b)

In [ ]:
slice_1[a] = result

In [ ]:
_contour_slices(slice_1)

In [ ]:
print(np.sum(slice_1))
origin_slice_1 = _edt[:,50,:]
print(np.sum(origin_slice_1))
